# Intergral Image

In an integral image the value of pixel (x,y) is equal to the sum of pixels above and to the left of (x,y)

<img src="src\integral_image.png" alt="Finding integral image" width="500"/>

### Integral image allows for calculation of sum of all pixels inside a given rectangle using only 4 corner values.

In below image, 

- Point 1 gives sum of all values in block A i.e integral image of Block A
- Point 2 gives sum of all values in block A and B i.e integral image of Block A + B
- Point 3 gives sum of all values in block A and C i.e integral image of Block A + C
- Point 4 gives sum of all values in block A,B,C and D i.e integral image of total block


<img src="src\integral_image_use.png" alt="Integral image use" width="500"/>



# Cascade Classifiers

Concatenation of a set of weak classifiers to create a strong classifier

- Weak classifiers - Limited performance
- Strong classifiers - robust performance

The Haar Classifier is a machine learning based approach, an algorithm created by Paul Viola and Michael Jones; which are trained from many many positive images (with faces) and negatives images (without faces)
- It starts by extracting Haar features from each image using windows show below

![Fig.1 Haar Windows](src/haar_windows.png "Haar Windows")


Each window is placed on the picture to calculate a single feature. 
- This feature is a single value obtained by subtracting the sum of pixels under the white part of the window from the sum of the pixels under the black part of the window.

Now, all possible sizes of each window are placed on all possible locations of each image to calculate plenty of features.

For example, in above image, we are extracting two features. 
- The first one focuses on the property that the region of the eyes is often darker than the area of the nose and cheeks. 
- The second feature relies on the property that the eyes are darker than the bridge of the nose.

But among all these features calculated, most of them are irrelevant. 

For example, when used on the cheek, the windows become irrelevant because none of these areas are darker or lighter than other regions on the cheeks, all sectors here are the same.

So we promptly discard irrelevant features and keep only those relevant using called Adaboost.

***In the end, the algorithm considers the fact that generally: most of the region in an image is a non-face region. Considering this, it’s a better idea to have a simple method to check if a window is a non-face region, and if it's not-face, discard it right away and don’t process it again. So we can focus mostly on the area where a face is.***


https://docs.opencv.org/2.4/modules/objdetect/doc/cascade_classification.html

https://www.youtube.com/watch?v=_QZLbR67fUU

OpenCV provides us with a class **cv2.CascadeClassifier** which takes as input the training file of the (Haar/LBP) classifier we want to load and loads it for us.

Since we want to load outhe Haar classifier,  its XML training files are stored in the opencv/data/haarcascades/ folder. You can also find them in the data folder of the Github repo

To detect face from an image using the CascadeClassifier we use **cv2.detectMultiScale** 

    cv2.detectMultiScale(image, scaleFactor, minNeighbors): 
    - This is a general function to detect objects, in this case, it'll detect faces since we called in the face cascade. 
        - If it finds a face, it returns a list of positions of said face in the form “Rect(x,y,w,h).”, 
        - if not, then returns “None”.

    Image: 
     - The first input is the grayscale image. So make sure the image is in grayscale.

    scaleFactor: 
     - compensates a false perception in size that occurs when one face appears to be bigger than the other simply because it is closer to the camera.

    minNeighbors: 
    - a detection algorithm that uses a moving window to detect objects, it does so by defining how many objects are found near the current one before it can declare the face found.

These are usually used one, there are other parameters as well

> Tune these parameters according to the information about data.

## HAAR vs LBP

Each OpenCV face detection classifiers has its pros and cons, but the major differences are in accuracy and speed.

So, in case more accurate detections are required, Haar classifier is the way to go. This is more suitable in technology such as security systems or high-end stalking.

But the LBP classifier is faster, therefore, should be used in mobile applications or embedded systems.

Note - both classifiers work with gray scale images only

![Fig. 2 HAAR vs LBP](src/haar-vs-lbp.png "HAAR vs LBP")

https://www.superdatascience.com/opencv-face-detection/


The Local Binary Patterns, or LBP in short, also needs to be trained on hundreds of images. LBP is a visual/texture descriptor, and thankfully, our faces are also composed of micro visual patterns.

So, LBP features are extracted to form a feature vector that classifies a face from a non-face.

Each training image is divided into some blocks.
For each block, LBP looks at 9 pixels (3×3 window) at a time, and with a particular interest in the pixel located in the center of the window.
Then, it compares the central pixel value with every neighbor's pixel value under the 3×3 window. For each neighbor pixel that is greater than or equal to the center pixel, it sets its value to 1, and for the others, it sets them to 0.

After that, it reads the updated pixel values (which can be either 0 or 1) in a clockwise order and forms a binary number. Next, it converts the binary number into a decimal number, and that decimal number is the new value of the center pixel. We do this for every pixel in a block.

Then, it converts each block values into a histogram, so now we have gotten one histogram for each block in an image.

Finally, it concatenates these block histograms to form a one feature vector for one image, which contains all the features we are interested. So, this is how we extract LBP features from a picture.

